In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt 
import plotly.express as px
import plotly.graph_objects as go
import warnings
import shap

from datetime import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc

import xgboost as xgb

from pdpbox import pdp, info_plots
from pdpbox import pdp, info_plots

sns.set()

In [2]:
df=pd.read_csv('Fic.csv')
df2=pd.read_csv('file.csv')

In [3]:
df

,Year,Country,CO2 emissions (metric tons per capita),GDP (current US$),"Life expectancy at birth, total (years)","GNI per capita, Atlas method (current US$)","School enrollment, primary (% gross)","Population, total",Transport services (% of commercial service exports),"Transport services (% of service exports, BoP)",Public credit registry coverage (% of adults)
0,2011 [YR2011],United Kingdom,7.025425,2.659882e+12,80.951220,41060.0,104.425713,63258810,9.800114,9.687685,0
1,2012 [YR2012],United Kingdom,7.332471,2.704017e+12,80.904878,41690.0,106.246078,63700215,10.218725,10.105228,0
2,2013 [YR2013],United Kingdom,7.064903,2.783251e+12,81.004878,42730.0,107.112000,64128273,9.623713,9.509182,0
3,2014 [YR2014],United Kingdom,6.421598,3.065521e+12,81.304878,44400.0,102.255547,64602298,9.314380,9.210033,0
4,2015 [YR2015],United Kingdom,6.148545,2.932785e+12,80.956098,44340.0,101.925270,65116219,9.020134,8.921660,0
5,2016 [YR2016],United Kingdom,5.774742,2.693248e+12,81.156098,42940.0,101.335213,65611593,8.681399,8.594154,0
6,2017 [YR2017],United Kingdom,5.546266,2.662484e+12,81.256098,41280.0,101.154556,66058859,8.495493,8.414957,0
7,2018 [YR2018],United Kingdom,5.398708,2.857317e+12,81.256098,41740.0,100.995552,66460344,8.135844,8.066427,0
8,2019 [YR2019],United Kingdom,NaN,2.830814e+12,81.204878,42130.0,NaN,66836327,9.585110,9.488768,0
9,2020 [YR2020],United Kingdom,NaN,2.707744e+12,NaN,NaN,NaN,67215293,6.012522,5.948490,..


In [4]:
df.isna().sum()

Year                                                     0
Country                                                  0
CO2 emissions (metric tons per capita)                  10
GDP (current US$)                                        1
Life expectancy at birth, total (years)                  5
GNI per capita, Atlas method (current US$)               4
School enrollment, primary (% gross)                    18
Population, total                                        0
Transport services (% of commercial service exports)     0
Transport services (% of service exports, BoP)           0
Public credit registry coverage (% of adults)            0
dtype: int64

In [5]:
dscrb=df.describe(include='all')
dscrb

,Year,Country,CO2 emissions (metric tons per capita),GDP (current US$),"Life expectancy at birth, total (years)","GNI per capita, Atlas method (current US$)","School enrollment, primary (% gross)","Population, total",Transport services (% of commercial service exports),"Transport services (% of service exports, BoP)",Public credit registry coverage (% of adults)
count,50,50,40.000000,4.900000e+01,45.000000,46.000000,32.000000,5.000000e+01,50.000000,50.000000,50
unique,10,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16
top,2016 [YR2016],Germany,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
freq,5,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24
mean,NaN,NaN,9.068395,6.609153e+12,81.333751,46224.782609,101.997083,1.323944e+08,16.055189,15.782593,NaN
std,NaN,NaN,3.787854,6.245535e+12,1.721026,6546.077617,1.772107,9.811473e+07,6.138789,6.016684,NaN
min,NaN,NaN,4.592377,2.438208e+12,78.539024,37860.000000,99.165321,6.325881e+07,6.012522,5.948490,NaN
25%,NaN,NaN,5.717623,2.786503e+12,80.539024,41702.500000,101.114805,6.659223e+07,10.690475,10.419717,NaN
50%,NaN,NaN,9.079323,3.732743e+12,81.156098,44295.000000,101.873356,8.201764e+07,15.759198,15.722508,NaN
75%,NaN,NaN,9.684771,5.155717e+12,82.578049,48115.000000,102.561762,1.274028e+08,20.092515,19.773852,NaN


In [6]:
df=df.fillna(0)

In [7]:
df

,Year,Country,CO2 emissions (metric tons per capita),GDP (current US$),"Life expectancy at birth, total (years)","GNI per capita, Atlas method (current US$)","School enrollment, primary (% gross)","Population, total",Transport services (% of commercial service exports),"Transport services (% of service exports, BoP)",Public credit registry coverage (% of adults)
0,2011 [YR2011],United Kingdom,7.025425,2.659882e+12,80.951220,41060.0,104.425713,63258810,9.800114,9.687685,0
1,2012 [YR2012],United Kingdom,7.332471,2.704017e+12,80.904878,41690.0,106.246078,63700215,10.218725,10.105228,0
2,2013 [YR2013],United Kingdom,7.064903,2.783251e+12,81.004878,42730.0,107.112000,64128273,9.623713,9.509182,0
3,2014 [YR2014],United Kingdom,6.421598,3.065521e+12,81.304878,44400.0,102.255547,64602298,9.314380,9.210033,0
4,2015 [YR2015],United Kingdom,6.148545,2.932785e+12,80.956098,44340.0,101.925270,65116219,9.020134,8.921660,0
5,2016 [YR2016],United Kingdom,5.774742,2.693248e+12,81.156098,42940.0,101.335213,65611593,8.681399,8.594154,0
6,2017 [YR2017],United Kingdom,5.546266,2.662484e+12,81.256098,41280.0,101.154556,66058859,8.495493,8.414957,0
7,2018 [YR2018],United Kingdom,5.398708,2.857317e+12,81.256098,41740.0,100.995552,66460344,8.135844,8.066427,0
8,2019 [YR2019],United Kingdom,0.000000,2.830814e+12,81.204878,42130.0,0.000000,66836327,9.585110,9.488768,0
9,2020 [YR2020],United Kingdom,0.000000,2.707744e+12,0.000000,0.0,0.000000,67215293,6.012522,5.948490,..


In [8]:
df['Year']=df['Year'].str[:4]


In [9]:
df

,Year,Country,CO2 emissions (metric tons per capita),GDP (current US$),"Life expectancy at birth, total (years)","GNI per capita, Atlas method (current US$)","School enrollment, primary (% gross)","Population, total",Transport services (% of commercial service exports),"Transport services (% of service exports, BoP)",Public credit registry coverage (% of adults)
0,2011,United Kingdom,7.025425,2.659882e+12,80.951220,41060.0,104.425713,63258810,9.800114,9.687685,0
1,2012,United Kingdom,7.332471,2.704017e+12,80.904878,41690.0,106.246078,63700215,10.218725,10.105228,0
2,2013,United Kingdom,7.064903,2.783251e+12,81.004878,42730.0,107.112000,64128273,9.623713,9.509182,0
3,2014,United Kingdom,6.421598,3.065521e+12,81.304878,44400.0,102.255547,64602298,9.314380,9.210033,0
4,2015,United Kingdom,6.148545,2.932785e+12,80.956098,44340.0,101.925270,65116219,9.020134,8.921660,0
5,2016,United Kingdom,5.774742,2.693248e+12,81.156098,42940.0,101.335213,65611593,8.681399,8.594154,0
6,2017,United Kingdom,5.546266,2.662484e+12,81.256098,41280.0,101.154556,66058859,8.495493,8.414957,0
7,2018,United Kingdom,5.398708,2.857317e+12,81.256098,41740.0,100.995552,66460344,8.135844,8.066427,0
8,2019,United Kingdom,0.000000,2.830814e+12,81.204878,42130.0,0.000000,66836327,9.585110,9.488768,0
9,2020,United Kingdom,0.000000,2.707744e+12,0.000000,0.0,0.000000,67215293,6.012522,5.948490,..


In [10]:
df['Year']=df['Year'].apply(lambda x: datetime.strptime(x, '%Y'))

In [ ]:
def preprocess(dataset):
    X = pd.DataFrame()
    X = dataset[col]
    #X=dataset['Type_Jour']
    
    Y = dataset['GIPA']

    # Label encoding for categorical variables
    label_encoder = LabelEncoder()

    X['Sens'] = label_encoder.fit_transform(dataset['Sens'])
    X['Doublon'] = label_encoder.fit_transform(dataset['Doublon'])
    


    return X, Y